In [5]:
train = read.csv('./Stat441_KaggleProject/train_selected_adam.csv', header=TRUE)
test = read.csv('./Stat441_KaggleProject/test_selected_adam.csv', header=TRUE)

train$essential[train$essential == 0] <- 1
test$essential[test$essential == 0] <- 3
test$essential[test$essential == 1] <- 2
test$essential[test$essential == 2] <- 1
test$essential[test$essential == 3] <- 2



cont_cols = c(sapply(seq(45, 57), function(x) {paste("X4", toString(x), sep='')}), 'correlation', 'protein', 'chromosome')
# binary_cols = c('X94','X95','X96','X97','X298', 'X307', 'X342')
# num_cols = c(cont_cols, binary_cols)
num_cols = c(cont_cols)
train[, !names(train) %in% num_cols] <- lapply(train[, !names(train) %in% num_cols], as.factor)
test[, !names(test) %in% num_cols] <- lapply(test[, !names(test) %in% num_cols], as.factor)
# train$binarysum <- rowSums(train[binary_cols])
# train <- train[, !names(train) %in% binary_cols]
# test$binarysum <- rowSums(test[binary_cols])
# test <- test[, !names(test) %in% binary_cols]

train_size <- floor(0.7 * nrow(train))
train_ind <- sample(seq_len(nrow(train)), size = train_size, replace = FALSE)
valid_ind <- seq_len(nrow(train))[-train_ind]
train_ind <- sample(seq_len(length(train_ind)), size = floor(1.5 * train_size), replace = TRUE)
train_df <- train[train_ind, ]
valid_df <- train[valid_ind, ]
train_df$label <- factor(train_df$label)

In [7]:
library(randomForest)
rf_clf <-  randomForest(label ~ . - protein, data = train_df, ntrees=5000)

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.



In [8]:
rf_clf


Call:
 randomForest(formula = d1 ~ . - protein - label, data = train_df,      ntrees = 5000) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 8

        OOB estimate of  error rate: 2.69%
Confusion matrix:
      FALSE TRUE class.error
FALSE   722    4 0.005509642
TRUE     33  618 0.050691244

In [14]:
valid_preds <- predict(rf_clf, valid_df)
valid_preds <- factor(valid_preds, levels=levels(valid_df$label))
valid_l <- valid_preds == valid_df$label
sum(valid_l) / length(valid_l)

[1] 0.8426396

In [16]:
train_preds <- predict(rf_clf, train_df)
train_preds <- factor(train_preds, levels=levels(train_df$label))
train_l <- train_preds == train_df$label
sum(train_l) / length(train_l)

[1] 0.9389978

In [11]:
for (i in colnames(test)){
    n1 = levels(test[, i])
    n2 = levels(train[, i])
    if (length(n1) != length(n2) || sum(n1 != n2)){
#         print(i)
#         print(levels(test[, i]))
#         print(levels(train[, i]))
        test[, i] <- factor(test[, i], levels = levels(train[, i]))
    }
}

test_preds <- predict(rf_clf, test)
test$label <- test_preds
table(test_preds) / length(test_preds)

test_preds
          0           1           2           3           4           5 
0.538388626 0.256872038 0.008530806 0.132701422 0.001895735 0.011374408 
          6           7           8           9          10          11 
0.050236967 0.000000000 0.000000000 0.000000000 0.000000000 0.000000000 

In [12]:
result <- data.frame("protein" = test$protein, "label" = test$label)
result_agg <- aggregate(result,
                by = list(result$protein),
                FUN = function(x) tail(names(sort(table(x))), 1))
result_agg <- data.frame("protein" = result_agg$protein, "label" = result_agg$label)
table(result_agg$label) / length(result_agg$label)


          0           1           2           3           4           5 
0.524934383 0.338582677 0.023622047 0.078740157 0.005249344 0.013123360 
          6 
0.015748031 

In [8]:
colnames(result_agg) <- c('Key', 'Label')
write.csv(result_agg,'./result_Mar31.csv', row.names = FALSE, quote=FALSE)

In [ ]:
model1 , predict(model1, test)
test - test[was prediced to be true]
model2, predict(model2, test)
